In [1]:
import soccerdata as sd
from joblib import Parallel, delayed
import pandas as pd
from tqdm import tqdm

[03/09/24 20:50:17] INFO     No custom team name replacements found. You can configure these in       ]8;id=581376;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=904827;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py#87\87]8;;\
                             C:\Users\gfuen\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=50195;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=177976;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py#229\229]8;;\
                             C:\Users\gfuen\soccerdata\config\league_dict.json.                                    

In [7]:
print(sd.ESPN.available_leagues())

['ENG-Premier League', 'ESP-La Liga', 'FRA-Ligue 1', 'GER-Bundesliga', 'ITA-Serie A', 'NED-Eredivisie']


In [8]:
# Arreglar : ---
# En serie
league = "ESP-La Liga"
seasons = [i for i in range(2000, 2023)]
espn = sd.ESPN(leagues=league, seasons=seasons, no_cache=True)
schedule = espn.read_schedule()
schedule

[03/09/24 17:16:36] INFO     Saving cached data to C:\Users\gfuen\soccerdata\data\ESPN                ]8;id=785367;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=862005;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py#91\91]8;;\

c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py:498: UserWarning: Season id "2021" is ambiguous: interpreting as "20-21"
  warnings.warn(msg, stacklevel=1)


date  \
league      season game                                                                 
ESP-La Liga 0001   2000-09-30 Racing Santander-Real Madrid  2000-09-30 07:00:00+00:00   
                   2000-10-01 Alavés-Osasuna                2000-10-01 07:00:00+00:00   
                   2000-10-01 Athletic Club-Mallorca        2000-10-01 07:00:00+00:00   
                   2000-10-01 Deportivo La Coruña-Barcelona 2000-10-01 07:00:00+00:00   
                   2000-10-01 Espanyol-Real Sociedad        2000-10-01 07:00:00+00:00   
...                                                                               ...   
            2223   2023-06-04 Real Betis-Valencia           2023-06-04 19:00:00+00:00   
                   2023-06-04 Real Madrid-Athletic Club     2023-06-04 16:30:00+00:00   
                   2023-06-04 Real Sociedad-Sevilla         2023-06-04 16:30:00+00:00   
                   2023-06-04 Real Valladolid-Getafe        2023-06-04 19:00:00+00:00   
                   2023-06-04 Villarreal-Atlético Madrid    2023-06-04 16:30:00+00:00   

                                                                       home_team  \
league      season game                                                            
ESP-La Liga 0001   2000-09-30 Racing Santander-Real Madrid      Racing Santander   
                   2000-10-01 Alavés-Osasuna                              Alavés   
                   2000-10-01 Athletic Club-Mallorca               Athletic Club   
                   2000-10-01 Deportivo La Coruña-Barcelona  Deportivo La Coruña   
                   2000-10-01 Espanyol-Real Sociedad                    Espanyol   
...                                                                          ...   
            2223   2023-06-04 Real Betis-Valencia                     Real Betis   
                   2023-06-04 Real Madrid-Athletic Club              Real Madrid   
                   2023-06-04 Real Sociedad-Sevilla                Real Sociedad   
                   2023-06-04 Real Valladolid-Getafe             Real Valladolid   
                   2023-06-04 Villarreal-Atlético Madrid              Villarreal   

                                                                   away_team  \
league      season game                                                        
ESP-La Liga 0001   2000-09-30 Racing Santander-Real Madrid       Real Madrid   
                   2000-10-01 Alavés-Osasuna                         Osasuna   
                   2000-10-01 Athletic Club-Mallorca                Mallorca   
                   2000-10-01 Deportivo La Coruña-Barcelona        Barcelona   
                   2000-10-01 Espanyol-Real Sociedad           Real Sociedad   
...                                                                      ...   
            2223   2023-06-04 Real Betis-Valencia                   Valencia   
                   2023-06-04 Real Madrid-Athletic Club        Athletic Club   
                   2023-06-04 Real Sociedad-Sevilla                  Sevilla   
                   2023-06-04 Real Valladolid-Getafe                  Getafe   
                   2023-06-04 Villarreal-Atlético Madrid     Atlético Madrid   

                                                             game_id league_id  
league      season game                                                         
ESP-La Liga 0001   2000-09-30 Racing Santander-Real Madrid      1177     esp.1  
                   2000-10-01 Alavés-Osasuna                    1183     esp.1  
                   2000-10-01 Athletic Club-Mallorca            1185     esp.1  
                   2000-10-01 Deportivo La Coruña-Barcelona     1178     esp.1  
                   2000-10-01 Espanyol-Real Sociedad            1184     esp.1  
...                                                              ...       ...  
            2223   2023-06-04 Real Betis-Valencia             644146     esp.1  
                   2023-06-04 Real Madrid-Athletic Club     

In [13]:
ids_con_problemas = [14654,14653,15868,16446,17917,17924,53989,56397]
leagues = sd.ESPN.available_leagues()

for l in leagues[1:]:
    seasons = [i for i in range(2000, 2023)]
    espn = sd.ESPN(leagues=l, seasons=seasons)
    schedule = espn.read_schedule()
    while True:
        try:
            match_ids = list(set(schedule.game_id.unique())- set(ids_con_problemas))
            df = espn.read_matchsheet(match_ids)
            df.to_csv(f"ESPN_{l}.csv")
            print(f"Liga {l} lista")
            break
        except ConnectionError as err:
            print(f"La liga {l} presenta otro error")
            id_broken = str(err).split("=")[-1][:-1]
            ids_con_problemas.append(int(id_broken))
        except:
            print(f"La liga {l} presenta otro error")
            break



[03/10/24 11:31:50] INFO     Saving cached data to C:\Users\gfuen\soccerdata\data\ESPN                ]8;id=327709;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=689375;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py#91\91]8;;\

KeyError: 'date'